# Cálculos preliminares de diseño

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Parámetros constantes
Tco=100 # Temperatura frío salida
Tci=25 # Temperatura frío entrada
dTc=np.abs(Tco-Tci)
dotmc=1500 # flujo de masa frío

Tho=25 # Temperatura caliente salida
Thi=140 # Temperatura caliente entrada
dTh=np.abs(Tho-Thi)
# dotmh=1300 # flujo de masa caliente

# parámetros de las placas

l=1242e-3 # m
L=3725e-3 # m
s=L*l
dp=2.5e-3 # m
kappa=60 # W/(m K)
# h=2500
G=dotmc/997 # m^3/s
vW=G/s # m/s
Prw=0.9
corr=30 # deg

In [3]:
dTh,dTc

(115, 75)

In [4]:
dTLM=(dTc-dTh)/np.log(dTc/dTh)

In [5]:
# Parámtros Leche
CpL=3893.72e-3

def DensidadLeche(T):
    # return 3.50e-2-3.58e-4*T+4.9e-6*T**2-1e-7*T**3
    return 1032.4

def ViscosidadLeche(T):
    return 1.236e-3

In [6]:
# Parámetros del agua
CpW=4186e-3

def DensidadAgua(T):
    return 997

def ViscosidadAgua(T):
    return 899.85e-6

In [7]:
def HeatFlux(Ti,To,dotm,Cp):
    return dotm*Cp*np.abs(To-Ti)

In [8]:
def Reynolds(rho,v,Dh,mu):
    return rho*v*Dh/mu

In [9]:
def DiámetroHidráulico(l,d):
    return 2*l*d/(l+d)

In [10]:
def Prandtl(mu,Cp,kappa):
    return mu*Cp/kappa*1000

In [11]:
def Nusselt(Re,Pr,corr):
    if corr<=30:
        if Re<=10:
            a=0.718
            b=0.349
        elif Re>10:
            a=0.348
            b=0.663
    elif corr==45:
        if Re<=10:
            a=0.718
            b=0.349
        elif 10<Re<=100:
            a=0.4
            b=0.598
        elif Re>100:
            a=0.3
            b=0.663
    elif corr==50:
        if Re<=20:
            a=0.630
            b=0.333
        elif 20<Re<=300:
            a=0.291
            b=0.591
        elif Re>300:
            a=0.130
            b=0.732
    elif corr==60:
        if Re<=20:
            a=0.562
            b=0.326
        elif 20<Re<=400:
            a=0.306
            b=0.529
        elif Re>400:
            a=0.108
            b=0.703
    elif corr>=65:
        if Re<=20:
            a=0.562
            b=0.326
        elif 20<Re<=500:
            a=0.331
            b=0.503
        elif Re>500:
            a=0.087
            b=0.718
            
    return a*Re**b*Pr**0.33*(Pr/Prw)**0.13

In [12]:
H=7 # Coeficiente de transferencia de calor total
P=100
p=0
while p<P:
    Phi=HeatFlux(Tci,Tco,dotmc,CpW)
    dotmh=Phi/CpL/np.abs(Tho-Thi)
    GL=dotmh/DensidadLeche(1)
    A=Phi/(H*dTLM)
    vL=GL/s
    N=A/s
    n=(N-1)/2
    hc=Nusselt(Reynolds(DensidadAgua(1),vW,DiámetroHidráulico(l,dp),ViscosidadAgua(1)),Prandtl(ViscosidadAgua(1),CpL,kappa),corr)*ViscosidadAgua(1)/DiámetroHidráulico(l,dp)
    hh=Nusselt(Reynolds(DensidadLeche(1),vL,DiámetroHidráulico(l,dp),ViscosidadLeche(1)),Prandtl(ViscosidadLeche(1),CpW,kappa),corr)*ViscosidadLeche(1)/DiámetroHidráulico(l,dp)
    Hestimado=1/(1/hc+0.48721+1/hh+0.1864+2e-2/kappa)
    error=np.abs(Hestimado-H)
    H=Hestimado
    p+=1

In [13]:
print("H:",H,"kW m^2/K \n N:",N,"\n n:",n,"\n hc:",hc,"kW m^2/K \n hh:",hh,"kW m^2/K \n dotmh:",dotmh,"kg/s \n Phi:",Phi,"kW/m^2 \n GL:",GL,"m^3/s \n vL:",vL,"m/s \n error:",error)

H: 0.6844034172797543 kW m^2/K 
 N: 1589.318553141897 
 n: 794.1592765709485 
 hc: 2.477707734664763 kW m^2/K 
 hh: 2.6069884745074154 kW m^2/K 
 dotmh: 1051.6934961938712 kg/s 
 Phi: 470925.0 kW/m^2 
 GL: 1.0186880048371474 m^3/s 
 vL: 0.22018783404924885 m/s 
 error: 0.0


# Caída de presión

In [14]:
def FactorFricción(corr,rho,v,mu):
    return (corr/30)**(0.83)*((30.2/Reynolds(rho,v,DiámetroHidráulico(l,dp),mu))**5+(6.28/Reynolds(rho,v,DiámetroHidráulico(l,dp),mu)**0.5))**0.2

In [15]:
def dPch(f,rho,v):
    return 2*f*rho*v**2*L/DiámetroHidráulico(l,dp)

In [16]:
def dPco(Np,dotm,Dco,rho):
    return 1.4*Np*(4*dotm/(np.pi*Dco**2))**2/(2*rho)*0.9**(-1.7)

In [17]:
dPchL=dPch(FactorFricción(corr,DensidadLeche(1),vL,ViscosidadLeche(1)),DensidadLeche(1),vL)
dPchW=dPch(FactorFricción(corr,DensidadAgua(1),vW,ViscosidadAgua(1)),DensidadAgua(1),vW)

In [18]:
print(dPchL,"Pa\n",dPchW,"Pa")

54552.442313076564 Pa
 107440.02990949758 Pa


In [19]:
dPcoL=dPco(n,dotmh,2,DensidadLeche(1))
dPcoW=dPco(n,dotmc,2,DensidadAgua(1))
print(dPcoL,"Pa\n",dPcoW,"Pa")

72181.18745165791 Pa
 152048.10143024253 Pa


In [20]:
dPL=dPchL+dPcoL
dPW=dPchW+dPcoW
print(dPL,"Pa\n",dPW,"Pa")

126733.62976473448 Pa
 259488.13133974012 Pa
